In [41]:
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os 
from datetime import datetime
import pandas as pd 
import numpy as np 
import keras
from sklearn.preprocessing import MinMaxScaler 
from sklearn.model_selection import TimeSeriesSplit 
from sklearn.model_selection import train_test_split
from keras.models import Sequential 
from keras.layers import Dense, Flatten, Dropout, LSTM, GRU
from sklearn.metrics import mean_squared_error
from keras.layers import Dropout,BatchNormalization

os.chdir(r'D:\Github 작업용\Github_cowork_practice')

In [42]:
train_data = pd.read_csv('Embedding_train_data.csv')
test_data = pd.read_csv('Embedding_test_data.csv')

In [43]:
train_data

,일자,요일,본사정원수,본사휴가자수,본사출장자수,본사시간외근무명령서승인건수,현본사소속재택근무자수,조식메뉴,중식메뉴,석식메뉴,...,석식91,석식92,석식93,석식94,석식95,석식96,석식97,석식98,석식99,석식100
0,2016-02-01,월,2601,50,150,238,0.0,"['모닝롤', '찐빵', '우유', '두유', '주스', '계란후라이', '호두죽'...","['쌀밥', '잡곡밥', '(쌀,현미흑미:국내산)', '오징어찌개', '쇠불고기',...","['쌀밥', '잡곡밥', '(쌀,현미흑미:국내산)', '육개장', '자반고등어구이'...",...,-0.801821,-0.380168,-0.619642,0.123337,-0.909220,0.419394,0.231539,0.452549,0.167723,0.397080
1,2016-02-02,화,2601,50,173,319,0.0,"['모닝롤', '단호박샌드', '우유', '두유', '주스', '계란후라이', '팥...","['쌀밥', '잡곡밥', '(쌀,현미흑미:국내산)', '김치찌개', '가자미튀김',...","['콩나물밥*양념장', '(쌀,현미흑미:국내산)', '어묵국', '유산슬', '(쇠...",...,-0.390211,-0.679858,-0.367707,0.231150,-0.119020,-0.072185,0.328293,0.571315,0.633374,-0.939822
2,2016-02-03,수,2601,56,180,111,0.0,"['모닝롤', '베이글', '우유', '두유', '주스', '계란후라이', '표고버...","['카레덮밥', '(쌀,현미흑미:국내산)', '팽이장국', '치킨핑거', '(닭고기...","['쌀밥', '잡곡밥', '(쌀,현미흑미:국내산)', '청국장찌개', '황태양념구이...",...,-0.010766,0.487713,-0.625857,0.548922,-0.326612,-1.426111,-0.141372,-0.274090,-0.276920,-0.127859
3,2016-02-04,목,2601,104,220,355,0.0,"['모닝롤', '토마토샌드', '우유', '두유', '주스', '계란후라이', '닭...","['쌀밥', '잡곡밥', '(쌀,현미흑미:국내산)', '쇠고기무국', '주꾸미볶음'...","['미니김밥*겨자장', '(쌀,현미흑미:국내산)', '우동', '멕시칸샐러드', '...",...,0.316081,-0.904823,-1.632920,-0.194064,-1.377587,-0.649722,-0.021026,0.662347,0.552243,-0.332059
4,2016-02-05,금,2601,278,181,34,0.0,"['모닝롤', '와플', '우유', '두유', '주스', '계란후라이', '쇠고기죽...","['쌀밥', '잡곡밥', '(쌀,현미흑미:국내산)', '떡국', '돈육씨앗강정', ...","['쌀밥', '잡곡밥', '(쌀,현미흑미:국내산)', '차돌박이찌개', '(쇠고기:...",...,0.487017,-0.463689,0.164248,0.137904,-0.841847,-0.223817,-0.516131,0.308813,-0.242846,-0.034809
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1200,2021-01-20,수,2983,75,198,4,391.0,"['모닝롤', '페퍼로니피자', '우유', '주스', '계란후라이', '찐계란', ...","['쌀밥', '흑미밥', '찰현미밥', '아욱국', '수제함박스테이크', '견과류마...","['김치볶음밥', '미니쫄우동*맛살튀김', '브로콜리깨소스무침', '계란후라이', ...",...,0.417278,0.237436,-0.223325,-0.819458,-1.515428,-0.034094,0.965614,1.173566,-0.050917,0.169004
1201,2021-01-21,목,2983,92,231,462,351.0,"['모닝롤', '생크림단팥빵', '우유', '주스', '계란후라이', '찐계란', ...","['쌀밥', '수수밥', '찰현미밥', '냉이된장국', '동파육', '봄동전', '...","['흑미밥', '쇠고기무국', '삼치양념구이', '비엔나채소볶음', '숙주나물당근무...",...,-0.052708,-0.074582,-0.349942,-0.702761,-0.519537,0.337701,0.702355,-0.465351,-0.837733,0.407873
1202,2021-01-22,금,2983,255,248,1,303.0,"['모닝롤', 'BLT샌드위치', '우유', '주스', '계란후라이', '찐계란',...","['전주비빔밥*약고추장', '계란파국', '요거닭', '올방개묵무침', '파프리카해...","['흑미밥', '수제비국', '수제맛쵸킹탕수육', '유부채소겨자냉채', '참나물무침...",...,0.252223,0.648180,-0.553343,-0.444173,-0.387879,-0.892189,0.174453,0.919008,0.633327,-1.756728
1203,2021-01-25,월,2983,107,153,616,327.0,"['모닝롤', '호박고구마오븐구이', '우유', '주스', '계란후라이', '찐계란...","['쌀밥', '흑미밥', '찰현미밥', '전주식콩나물해장국', '돈육간장불고기', ...","['흑미밥', '열무된장국', '장어강정*데리야끼소스', '깻잎쌈*생강채', '오이...",...,-0.874287,-0.033421,-0.612048,-0.211504,-0.143802,0.533571,0.611764,-0.347412,-0.916020,-0.891743


In [44]:
test_data

,일자,요일,본사정원수,본사휴가자수,본사출장자수,본사시간외근무명령서승인건수,현본사소속재택근무자수,조식메뉴,중식메뉴,석식메뉴,...,석식91,석식92,석식93,석식94,석식95,석식96,석식97,석식98,석식99,석식100
0,2021-01-27,수,2983,88,182,5,358.0,"['모닝롤', '연유버터베이글', '우유', '주스', '계란후라이', '찐계란',...","['쌀밥', '흑미밥', '찰현미밥', '대구지리', '매운돈갈비찜', '오꼬노미계...","['흑미밥', '얼큰순두부찌개', '쇠고기우엉볶음', '버섯햄볶음', '(New)아...",...,-0.374542,-0.287505,-0.520426,-0.530235,-0.941172,0.993017,1.466026,-0.602551,-0.505692,-0.735012
1,2021-01-28,목,2983,104,212,409,348.0,"['모닝롤', '대만샌드위치', '우유', '주스', '계란후라이', '찐계란', ...","['쌀밥', '보리밥', '찰현미밥', '우렁된장찌개', '오리주물럭', '청양부추...","['충무김밥', '우동국물', '오징어무침', '꽃맛살샐러드', '얼갈이쌈장무침',...",...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,2021-01-29,금,2983,270,249,0,294.0,"['모닝롤', '핫케익', '우유', '주스', '계란후라이', '찐계란', '오곡...","['쌀밥', '흑미밥', '찰현미밥', '팽이장국', '수제돈까스*소스', '가자미...","['흑미밥', '물만둣국', '카레찜닭', '숯불양념꼬지어묵', '꼬시래기무침', ...",...,-0.515465,-0.705520,-1.054055,-0.545785,-0.650279,0.397171,0.790346,-0.395544,-0.427766,-0.994062
3,2021-02-01,월,2924,108,154,538,322.0,"['모닝롤', '촉촉한치즈케익', '우유', '주스', '계란후라이', '찐계란',...","['쌀밥', '흑미밥', '찰현미밥', '배추들깨국', '오리대패불고기', '시금치...","['흑미밥', '동태탕', '돈육꽈리고추장조림', '당면채소무침', '모자반무침',...",...,-0.985826,-1.253553,-0.463552,-0.282734,-0.021294,0.000165,1.331887,-0.949383,-1.051228,-0.583881
4,2021-02-02,화,2924,62,186,455,314.0,"['모닝롤', '토마토샌드', '우유', '주스', '계란후라이', '찐계란', '...","['쌀밥', '팥밥', '찰현미밥', '부대찌개', '닭살데리야끼조림', '버섯탕수...","['흑미밥', '바지락살국', '쇠고기청경채볶음', '두부구이*볶은김치', '머위된...",...,0.773249,-0.749277,-0.948147,-1.684721,0.261702,0.822280,1.130417,-2.061396,0.060347,-1.256325
5,2021-02-03,수,2924,59,199,5,286.0,"['모닝롤', '게살모닝샌드', '우유', '주스', '계란후라이', '찐계란', ...","['쌀밥', '흑미밥', '찰현미밥', '아욱국', '매콤해물볶음', '감자조림',...","['오므라이스', '가쓰오장국', '빌소세지구이*구운채소', '단감치커리무침', '...",...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
6,2021-02-04,목,2924,61,211,476,288.0,"['모닝롤', '사과파이', '우유', '주스', '계란후라이', '찐계란', '누...","['쌀밥', '차조밥', '찰현미밥', '설렁탕', '고등어김치말이찜', '볼어묵굴...","['흑미밥', '계란파국', '돈육두루치기', '감자채파프리카볶음', '세발나물오리...",...,-0.515465,-0.705520,-1.054055,-0.545785,-0.650279,0.397171,0.790346,-0.395544,-0.427766,-0.994062
7,2021-02-05,금,2924,169,252,0,256.0,"['모닝롤', '앙버터모닝빵', '우유', '주스', '계란후라이', '찐계란', ...","['쌀밥', '흑미밥', '찰현미밥', '북엇국', '닭볶음탕', '채소전*장', ...","['유부초밥', '추가밥', '온메밀소바', '국물떡볶이', '순대찜*소금', '청...",...,-0.333308,-1.475864,-0.714185,1.483960,-0.850368,-0.469901,1.027584,0.696060,-0.891219,-0.302799
8,2021-02-08,월,2924,88,174,690,329.0,"['모닝롤', '스콘', '우유', '주스', '계란후라이', '찐계란', '누룽지...","['쌀밥', '흑미밥', '찰현미밥', '감자양파국', '돈수육*씨앗쌈장', '매콤...","['흑미밥', '냉이국', '반반치킨', '꼬막채소무침', '청경채찜', '포기김치']",...,-0.090330,-0.149433,-0.026438,-0.077751,0.429910,0.108399,0.931176,-0.918079,-0.036475,-0.168863
9,2021-02-09,화,2924,94,183,542,329.0,"['모닝롤', '치즈팡샌드', '우유', '주스', '계란후라이', '찐계란', '...","['쌀밥', '기장밥', '찰현미밥', '장각백숙', '적어양념장구이', '채소스틱...","['흑미밥', '미역국', '매운소불고기', '단호박두부탕수', '메추리알장조림',...",...,0.163890,1.037070,-0.483276,-0.610491,1.054497,0.647570,0.047814,-0.446598,0.053169,-0.361961


# 전처리 실시한다

일자의 '월', '일' 만 사용할 예정이며 조식/중식/석식 메뉴는 제거한다.

In [45]:
train_data = train_data.drop(['조식메뉴', '중식메뉴', '석식메뉴'], axis = 1)
test_data = test_data.drop(['조식메뉴', '중식메뉴', '석식메뉴'], axis = 1)

In [46]:
train_data['월'] = train_data.일자.str[5:7]
train_data['일'] = train_data.일자.str[8:10]

test_data['월'] = train_data.일자.str[5:7]
test_data['일'] = train_data.일자.str[8:10]

In [47]:
train_data

,일자,요일,본사정원수,본사휴가자수,본사출장자수,본사시간외근무명령서승인건수,현본사소속재택근무자수,중식계,석식계,조식1,...,석식93,석식94,석식95,석식96,석식97,석식98,석식99,석식100,월,일
0,2016-02-01,월,2601,50,150,238,0.0,1039.0,331.0,0.127304,...,-0.619642,0.123337,-0.909220,0.419394,0.231539,0.452549,0.167723,0.397080,02,01
1,2016-02-02,화,2601,50,173,319,0.0,867.0,560.0,0.403453,...,-0.367707,0.231150,-0.119020,-0.072185,0.328293,0.571315,0.633374,-0.939822,02,02
2,2016-02-03,수,2601,56,180,111,0.0,1017.0,573.0,0.216448,...,-0.625857,0.548922,-0.326612,-1.426111,-0.141372,-0.274090,-0.276920,-0.127859,02,03
3,2016-02-04,목,2601,104,220,355,0.0,978.0,525.0,-0.192419,...,-1.632920,-0.194064,-1.377587,-0.649722,-0.021026,0.662347,0.552243,-0.332059,02,04
4,2016-02-05,금,2601,278,181,34,0.0,925.0,330.0,0.064057,...,0.164248,0.137904,-0.841847,-0.223817,-0.516131,0.308813,-0.242846,-0.034809,02,05
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1200,2021-01-20,수,2983,75,198,4,391.0,1093.0,421.0,-2.034448,...,-0.223325,-0.819458,-1.515428,-0.034094,0.965614,1.173566,-0.050917,0.169004,01,20
1201,2021-01-21,목,2983,92,231,462,351.0,832.0,353.0,-1.324221,...,-0.349942,-0.702761,-0.519537,0.337701,0.702355,-0.465351,-0.837733,0.407873,01,21
1202,2021-01-22,금,2983,255,248,1,303.0,579.0,217.0,-1.732236,...,-0.553343,-0.444173,-0.387879,-0.892189,0.174453,0.919008,0.633327,-1.756728,01,22
1203,2021-01-25,월,2983,107,153,616,327.0,1145.0,502.0,-1.805069,...,-0.612048,-0.211504,-0.143802,0.533571,0.611764,-0.347412,-0.916020,-0.891743,01,25


In [48]:
test_data

,일자,요일,본사정원수,본사휴가자수,본사출장자수,본사시간외근무명령서승인건수,현본사소속재택근무자수,조식1,조식2,조식3,...,석식93,석식94,석식95,석식96,석식97,석식98,석식99,석식100,월,일
0,2021-01-27,수,2983,88,182,5,358.0,-1.821336,0.237686,-0.973366,...,-0.520426,-0.530235,-0.941172,0.993017,1.466026,-0.602551,-0.505692,-0.735012,02,01
1,2021-01-28,목,2983,104,212,409,348.0,-1.209769,-0.392556,-0.580457,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,02,02
2,2021-01-29,금,2983,270,249,0,294.0,-1.423517,-0.344230,-0.590078,...,-1.054055,-0.545785,-0.650279,0.397171,0.790346,-0.395544,-0.427766,-0.994062,02,03
3,2021-02-01,월,2924,108,154,538,322.0,-1.581092,0.041783,-0.683824,...,-0.463552,-0.282734,-0.021294,0.000165,1.331887,-0.949383,-1.051228,-0.583881,02,04
4,2021-02-02,화,2924,62,186,455,314.0,-1.662282,-0.023853,-1.222977,...,-0.948147,-1.684721,0.261702,0.822280,1.130417,-2.061396,0.060347,-1.256325,02,05
5,2021-02-03,수,2924,59,199,5,286.0,-1.717155,-0.307514,-1.028005,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,02,11
6,2021-02-04,목,2924,61,211,476,288.0,-1.646292,-0.386000,-0.508134,...,-1.054055,-0.545785,-0.650279,0.397171,0.790346,-0.395544,-0.427766,-0.994062,02,12
7,2021-02-05,금,2924,169,252,0,256.0,-1.550466,0.079913,-0.925992,...,-0.714185,1.483960,-0.850368,-0.469901,1.027584,0.696060,-0.891219,-0.302799,02,15
8,2021-02-08,월,2924,88,174,690,329.0,-1.484573,-0.453224,-0.475515,...,-0.026438,-0.077751,0.429910,0.108399,0.931176,-0.918079,-0.036475,-0.168863,02,16
9,2021-02-09,화,2924,94,183,542,329.0,-1.787411,-0.689851,-0.867506,...,-0.483276,-0.610491,1.054497,0.647570,0.047814,-0.446598,0.053169,-0.361961,02,17


일자는 이제 안쓸거니까 갖다 버린다.

In [49]:
train_data = train_data.drop(['일자'], axis = 1)
test_data = test_data.drop(['일자'], axis = 1)

범주형은 단 하나, 요일 변수가 존재한다.

In [50]:
from sklearn.preprocessing import OneHotEncoder

o_scaler = OneHotEncoder(sparse=False)

train_lab = o_scaler.fit_transform(train_data[['요일']])
test_lab = o_scaler.transform(test_data[['요일']])

train_lab = pd.DataFrame(train_lab, columns = ['요일1','요일2','요일3','요일4','요일5'])
test_lab = pd.DataFrame(test_lab, columns = ['요일1','요일2','요일3','요일4','요일5'])

train_data = pd.concat([train_data, train_lab], axis = 1)
test_data = pd.concat([test_data, test_lab], axis = 1)

In [51]:
train_data

,요일,본사정원수,본사휴가자수,본사출장자수,본사시간외근무명령서승인건수,현본사소속재택근무자수,중식계,석식계,조식1,조식2,...,석식98,석식99,석식100,월,일,요일1,요일2,요일3,요일4,요일5
0,월,2601,50,150,238,0.0,1039.0,331.0,0.127304,0.257607,...,0.452549,0.167723,0.397080,02,01,0.0,0.0,0.0,1.0,0.0
1,화,2601,50,173,319,0.0,867.0,560.0,0.403453,0.024484,...,0.571315,0.633374,-0.939822,02,02,0.0,0.0,0.0,0.0,1.0
2,수,2601,56,180,111,0.0,1017.0,573.0,0.216448,0.308116,...,-0.274090,-0.276920,-0.127859,02,03,0.0,0.0,1.0,0.0,0.0
3,목,2601,104,220,355,0.0,978.0,525.0,-0.192419,0.578234,...,0.662347,0.552243,-0.332059,02,04,0.0,1.0,0.0,0.0,0.0
4,금,2601,278,181,34,0.0,925.0,330.0,0.064057,0.565667,...,0.308813,-0.242846,-0.034809,02,05,1.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1200,수,2983,75,198,4,391.0,1093.0,421.0,-2.034448,-0.353786,...,1.173566,-0.050917,0.169004,01,20,0.0,0.0,1.0,0.0,0.0
1201,목,2983,92,231,462,351.0,832.0,353.0,-1.324221,-0.470396,...,-0.465351,-0.837733,0.407873,01,21,0.0,1.0,0.0,0.0,0.0
1202,금,2983,255,248,1,303.0,579.0,217.0,-1.732236,-0.079323,...,0.919008,0.633327,-1.756728,01,22,1.0,0.0,0.0,0.0,0.0
1203,월,2983,107,153,616,327.0,1145.0,502.0,-1.805069,-0.131774,...,-0.347412,-0.916020,-0.891743,01,25,0.0,0.0,0.0,1.0,0.0


In [52]:
test_data

,요일,본사정원수,본사휴가자수,본사출장자수,본사시간외근무명령서승인건수,현본사소속재택근무자수,조식1,조식2,조식3,조식4,...,석식98,석식99,석식100,월,일,요일1,요일2,요일3,요일4,요일5
0,수,2983,88,182,5,358.0,-1.821336,0.237686,-0.973366,-0.736827,...,-0.602551,-0.505692,-0.735012,02,01,0.0,0.0,1.0,0.0,0.0
1,목,2983,104,212,409,348.0,-1.209769,-0.392556,-0.580457,-0.756506,...,0.000000,0.000000,0.000000,02,02,0.0,1.0,0.0,0.0,0.0
2,금,2983,270,249,0,294.0,-1.423517,-0.344230,-0.590078,-0.400957,...,-0.395544,-0.427766,-0.994062,02,03,1.0,0.0,0.0,0.0,0.0
3,월,2924,108,154,538,322.0,-1.581092,0.041783,-0.683824,-0.687377,...,-0.949383,-1.051228,-0.583881,02,04,0.0,0.0,0.0,1.0,0.0
4,화,2924,62,186,455,314.0,-1.662282,-0.023853,-1.222977,-0.857430,...,-2.061396,0.060347,-1.256325,02,05,0.0,0.0,0.0,0.0,1.0
5,수,2924,59,199,5,286.0,-1.717155,-0.307514,-1.028005,-0.769036,...,0.000000,0.000000,0.000000,02,11,0.0,0.0,1.0,0.0,0.0
6,목,2924,61,211,476,288.0,-1.646292,-0.386000,-0.508134,-0.276603,...,-0.395544,-0.427766,-0.994062,02,12,0.0,1.0,0.0,0.0,0.0
7,금,2924,169,252,0,256.0,-1.550466,0.079913,-0.925992,-1.202348,...,0.696060,-0.891219,-0.302799,02,15,1.0,0.0,0.0,0.0,0.0
8,월,2924,88,174,690,329.0,-1.484573,-0.453224,-0.475515,-0.466826,...,-0.918079,-0.036475,-0.168863,02,16,0.0,0.0,0.0,1.0,0.0
9,화,2924,94,183,542,329.0,-1.787411,-0.689851,-0.867506,-1.135271,...,-0.446598,0.053169,-0.361961,02,17,0.0,0.0,0.0,0.0,1.0


In [53]:
train_data = train_data.drop(['요일'], axis = 1)
test_data = test_data.drop(['요일'], axis = 1)

최종 input 형태는 다음과 같다.

In [54]:
train_data.head()

,본사정원수,본사휴가자수,본사출장자수,본사시간외근무명령서승인건수,현본사소속재택근무자수,중식계,석식계,조식1,조식2,조식3,...,석식98,석식99,석식100,월,일,요일1,요일2,요일3,요일4,요일5
0,2601,50,150,238,0.0,1039.0,331.0,0.127304,0.257607,0.547612,...,0.452549,0.167723,0.397080,02,01,0.0,0.0,0.0,1.0,0.0
1,2601,50,173,319,0.0,867.0,560.0,0.403453,0.024484,0.247226,...,0.571315,0.633374,-0.939822,02,02,0.0,0.0,0.0,0.0,1.0
2,2601,56,180,111,0.0,1017.0,573.0,0.216448,0.308116,0.311472,...,-0.274090,-0.276920,-0.127859,02,03,0.0,0.0,1.0,0.0,0.0
3,2601,104,220,355,0.0,978.0,525.0,-0.192419,0.578234,-0.261870,...,0.662347,0.552243,-0.332059,02,04,0.0,1.0,0.0,0.0,0.0
4,2601,278,181,34,0.0,925.0,330.0,0.064057,0.565667,0.058803,...,0.308813,-0.242846,-0.034809,02,05,1.0,0.0,0.0,0.0,0.0


이제 train / train_target / test의 분할을 실시한다.

In [55]:
train_target = train_data.loc[:,['중식계','석식계']]
train_data = train_data.drop(['중식계','석식계'],axis=1)

In [56]:
#정규화
# onehot부분이 중간에 중복으로 들어가긴 했지만, 어짜피 값 변화가 없으므로 그냥 그대로 진행한다.
data_scaler = MinMaxScaler()
data_scaled = data_scaler.fit_transform(train_data)

target_scaler = MinMaxScaler()
target_scaled = target_scaler.fit_transform(train_target)


In [57]:
# train set, test set 분할 
x_train, x_valid, y_train, y_valid = train_test_split(data_scaled, target_scaled, test_size=0.2, shuffle=True, random_state = 1724)

In [58]:
x_train.shape

(964, 312)

In [110]:
model = Sequential()
model.add(Dense(512, input_shape=(312,), activation='swish'))
model.add(Dropout(0.25))
model.add(Dense(512,activation='swish'))
model.add(Dropout(0.25))
model.add(Dense(256,activation='swish'))
model.add(Dropout(0.25))
model.add(Dense(256,activation='swish'))
model.add(Dropout(0.25))
model.add(Dense(2))

# model compile
model.compile(loss='mse', optimizer='adam')
# SGD, adam, rmsprop 셋중 하나 고르자.
# 너무 깊어서 adam보다 sgd가 학습을 잘해준다.

model.summary()

Model: "sequential_9"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_45 (Dense)            (None, 512)               160256    
                                                                 
 dropout_36 (Dropout)        (None, 512)               0         
                                                                 
 dense_46 (Dense)            (None, 512)               262656    
                                                                 
 dropout_37 (Dropout)        (None, 512)               0         
                                                                 
 dense_47 (Dense)            (None, 256)               131328    
                                                                 
 dropout_38 (Dropout)        (None, 256)               0         
                                                                 
 dense_48 (Dense)            (None, 256)              

In [111]:
# model setting
early_stopping = keras.callbacks.EarlyStopping(patience = 30, restore_best_weights = True)
reduce_lr = keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=15, min_lr=0.000001)
history = model.fit(x_train, y_train, batch_size=256, epochs=5000, callbacks=[early_stopping, reduce_lr], validation_data=(x_valid, y_valid))

Epoch 1/5000
4/4 [==============================] - 0s 31ms/step - loss: 0.1337 - val_loss: 0.0634 - lr: 0.0010
Epoch 2/5000
4/4 [==============================] - 0s 8ms/step - loss: 0.0528 - val_loss: 0.0381 - lr: 0.0010
Epoch 3/5000
4/4 [==============================] - 0s 8ms/step - loss: 0.0420 - val_loss: 0.0331 - lr: 0.0010
Epoch 4/5000
4/4 [==============================] - 0s 8ms/step - loss: 0.0351 - val_loss: 0.0323 - lr: 0.0010
Epoch 5/5000
4/4 [==============================] - 0s 8ms/step - loss: 0.0318 - val_loss: 0.0284 - lr: 0.0010
Epoch 6/5000
4/4 [==============================] - 0s 8ms/step - loss: 0.0304 - val_loss: 0.0259 - lr: 0.0010
Epoch 7/5000
4/4 [==============================] - 0s 9ms/step - loss: 0.0282 - val_loss: 0.0220 - lr: 0.0010
Epoch 8/5000
4/4 [==============================] - 0s 8ms/step - loss: 0.0253 - val_loss: 0.0219 - lr: 0.0010
Epoch 9/5000
4/4 [==============================] - 0s 8ms/step - loss: 0.0252 - val_loss: 0.0201 - lr: 0.0010


In [112]:
model.save('nn_model.h5')
# model = keras.models.load_model('nn_model.h5')

In [113]:
#train 예측 
train_predictions = model.predict(x_train)

#train 역변환 
val = target_scaler.inverse_transform(y_train)
predictions = target_scaler.inverse_transform(train_predictions)

#train RMSE
train_RMSE = mean_squared_error(val, predictions)**0.5

#test 예측 
valid_predictions = model.predict(x_valid)

#test 역변환 
val = target_scaler.inverse_transform(y_valid)
predictions = target_scaler.inverse_transform(valid_predictions)

#test RMSE
valid_RMSE = mean_squared_error(val, predictions)**0.5

print('train_RMSE:', train_RMSE)
print('valid_RMSE:', valid_RMSE)

train_RMSE: 75.79693943306242
valid_RMSE: 95.88805690509939


성능평가 마쳤으면 그냥 원래 test셋으로 submisson 준비해야지

In [114]:
test_scaled = data_scaler.transform(test_data)

test_predictions = model.predict(test_scaled)

test_predictions.shape

(50, 2)

In [115]:
y_pred = target_scaler.inverse_transform(test_predictions)
y_pred

array([[1053.5618 ,  344.12784],
       [ 962.32196,  421.01675],
       [ 691.42267,  342.41937],
       [1200.2736 ,  517.28326],
       [1003.69794,  458.4228 ],
       [1006.64685,  343.3781 ],
       [ 981.1589 ,  446.1297 ],
       [ 732.2142 ,  334.04837],
       [1310.7406 ,  576.93066],
       [ 963.86725,  405.45566],
       [ 822.4217 ,  204.72992],
       [1270.6482 ,  556.18005],
       [1087.012  ,  557.12476],
       [1059.2772 ,  304.72183],
       [ 854.82916,  438.27945],
       [ 668.18726,  281.39432],
       [1254.6213 ,  627.2608 ],
       [1122.5009 ,  637.2917 ],
       [ 968.62317,  458.78082],
       [ 923.15686,  487.11435],
       [ 707.85   ,  389.42477],
       [1101.6323 ,  564.50726],
       [ 997.62225,  433.19144],
       [1007.9861 ,  554.65826],
       [ 716.1632 ,  378.15576],
       [1302.9772 ,  651.7903 ],
       [1210.7362 ,  653.6841 ],
       [ 919.6773 ,  301.3014 ],
       [ 951.29895,  463.79416],
       [ 762.8223 ,  373.73627],
       [13

In [116]:
y_pred

array([[1053.5618 ,  344.12784],
       [ 962.32196,  421.01675],
       [ 691.42267,  342.41937],
       [1200.2736 ,  517.28326],
       [1003.69794,  458.4228 ],
       [1006.64685,  343.3781 ],
       [ 981.1589 ,  446.1297 ],
       [ 732.2142 ,  334.04837],
       [1310.7406 ,  576.93066],
       [ 963.86725,  405.45566],
       [ 822.4217 ,  204.72992],
       [1270.6482 ,  556.18005],
       [1087.012  ,  557.12476],
       [1059.2772 ,  304.72183],
       [ 854.82916,  438.27945],
       [ 668.18726,  281.39432],
       [1254.6213 ,  627.2608 ],
       [1122.5009 ,  637.2917 ],
       [ 968.62317,  458.78082],
       [ 923.15686,  487.11435],
       [ 707.85   ,  389.42477],
       [1101.6323 ,  564.50726],
       [ 997.62225,  433.19144],
       [1007.9861 ,  554.65826],
       [ 716.1632 ,  378.15576],
       [1302.9772 ,  651.7903 ],
       [1210.7362 ,  653.6841 ],
       [ 919.6773 ,  301.3014 ],
       [ 951.29895,  463.79416],
       [ 762.8223 ,  373.73627],
       [13

In [117]:
submission = pd.read_csv('sample_submission.csv')

In [118]:
submission['중식계'] = y_pred[:,0]
submission['석식계'] = y_pred[:,1]

In [119]:
submission

,일자,중식계,석식계
0,2021-01-27,1053.561768,344.127838
1,2021-01-28,962.321960,421.016754
2,2021-01-29,691.422668,342.419373
3,2021-02-01,1200.273560,517.283264
4,2021-02-02,1003.697937,458.422791
5,2021-02-03,1006.646851,343.378113
6,2021-02-04,981.158875,446.129700
7,2021-02-05,732.214172,334.048370
8,2021-02-08,1310.740601,576.930664
9,2021-02-09,963.867249,405.455658


식수 계는 소수점이 없으므로 이것도 제출해본다.

In [120]:
submission1 = submission.round(0)

In [121]:
submission1

,일자,중식계,석식계
0,2021-01-27,1054.0,344.0
1,2021-01-28,962.0,421.0
2,2021-01-29,691.0,342.0
3,2021-02-01,1200.0,517.0
4,2021-02-02,1004.0,458.0
5,2021-02-03,1007.0,343.0
6,2021-02-04,981.0,446.0
7,2021-02-05,732.0,334.0
8,2021-02-08,1311.0,577.0
9,2021-02-09,964.0,405.0


In [122]:
submission.to_csv('submission_NNmodel.csv', index = False)
submission1.to_csv('submission_NNmodel_round.csv', index = False)